<a href="https://colab.research.google.com/github/ryooan/metaculus-bot-forecaster/blob/main/Ryan_LLM_Forecast_Bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LLM Forecasting Bot
*Created by Kirill and Tom, revised by Ryan*

The code below is used for making LLM-powered forecasts in Metaculus' [AI benchmarking competition](https://www.metaculus.com/project/ai-benchmarking-pilot/).

Specifically, it does the following:

* Gets questions from the project page using the Metaculus API
* Gets four separate forecasts from the LLM, three independently and the fourth assessing the reasoning of the first three and producing its own.
* Predicts on the Metaculus questions and shares a comment describing the reasoning of the fourth LLM forecast.

Features and options:

* Allows you to choose whether it repredicts on questions it's already forecasted on or ignores them.
* Allows for the use of [Perplexity search](https://www.perplexity.ai/) for additional research, using a prompt formed by an LLM.
    * *Previously this allowed for the use of pre-computed Perpelxity results, but this is no longer supported by Metaculus.*
* Can be used with an automated workflow via Github actions to monitor the project for new open questions and make forecasts when there are some
    * See [this Github repo](https://github.com/ryooan/metaculus-bot-forecaster) for how to set this up

### 🚨🚨🚨 Warning 🚨🚨🚨

**You are responsible for monitoring the costs of your implementation, especially if using the automated Github workflow. Cost estimates computed by this notebook are rough estimates only, make sure to check and monitor how much you are spending and the funds in your relevant accounts.**

## Getting Started

### Make a Metaculus Bot Account
The first step will be to make a Metaculus bot account. Instructions for how to do this have likely already been provided to you, either via a page on Metaculus or at an event.

### Secrets and Tokens

You need to set secrets 1) and 2) in order to make forecasts. Secrets 3) and 4) are necessary if you will be using Perplexity research.

1) METACULUS_TOKEN (you can find it or create it here - https://www.metaculus.com/admin/authtoken/tokenproxy/, or ask Metaculus to share it with you).

2) OPENAPI_API_KEY - (you can find it here https://platform.openai.com/settings/profile?tab=api-keys).

3) PERPLEXITY_API_KEY - You can generate an API key here: https://docs.perplexity.ai/docs/getting-started

These secrets can be set in your Google colab account using the key on the left side.

*See the [Github repo](https://github.com/ryooan/metaculus-bot-forecaster) for special instructions necessary for setting your secrets in Github if you intend to use the automated Github action.*

*Note: previously this also used a QUESTIONS_API_KEY which got some precomputed Perplexity results stored by Metaculus, but this is no longer supported by Metaculus.*

### Setting Inputs

Once your tokens are set correctly, you can proceed to the [Inputs section](https://colab.research.google.com/drive/1_P7_QNJiJyWBY2qCVu2-_8gVPD1X7mX3?authuser=2#scrollTo=6cbruBaVtaZh). That should be the only section most users will need. More advanced users can edit the [Setup](https://colab.research.google.com/drive/1_P7_QNJiJyWBY2qCVu2-_8gVPD1X7mX3?authuser=2#scrollTo=tNl_mbJaX60R) and [Code](https://colab.research.google.com/drive/1_P7_QNJiJyWBY2qCVu2-_8gVPD1X7mX3?authuser=2#scrollTo=k8vtze4SXtR3) sections if desired, but this is not recommended unless you have coding experience.

## Setup

It is recommended that you do not edit these cells unless you have coding experience.



---



In [21]:
from IPython.display import HTML, display

def set_css(*args, **kwargs):
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

#according to this link the above wraps the output text: https://stackoverflow.com/questions/58890109/line-wrapping-in-collaboratory-google-results

In [22]:
!pip install openai
!pip install tiktoken
import datetime
import json
import os
import requests
import tiktoken
import re

from openai import OpenAI

#use the below to detect if it's being run in google colab, if it's not this skips an error
def in_colab():
    try:
        import google.colab
        return True
    except ImportError:
        return False

if in_colab():
    from google.colab import userdata

In [23]:
#the below is used to get secrets when using github actions to automate
#initialize
token = None
questions_api_key = None
perplexity_api_key = None

# Function to load secrets from the specified path
def load_secrets(secrets_path):
    try:
        with open(secrets_path, 'r') as secrets_file:
            secrets = json.loads(secrets_file.read())
            for k, v in secrets.items():
                os.environ[k] = v
    except Exception as e:
        print(f"Error loading secrets from {secrets_path}: {e}")

# Main code block
try:
    if 'secretsPath' in globals():
        print(f"secretsPath exists: {secretsPath}")
        load_secrets(secretsPath)

        token = os.environ['METACULUS_TOKEN']
        questions_api_key = os.environ['QUESTIONS_API_KEY']
        perplexity_api_key = os.environ['PERPLEXITY_API_KEY']
    else:
        raise NameError("secretsPath not defined")
except NameError:
    print("Loading secrets from userdata (Google Colab)")
    token = userdata.get('METACULUS_TOKEN')
    os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
    questions_api_key = userdata.get('QUESTIONS_API_KEY')
    perplexity_api_key = userdata.get('PERPLEXITY_API_KEY')
except KeyError as e:
    print(f"Missing required environment variable: {e}")

Loading secrets from userdata (Google Colab)


## Inputs

The cell below contains all of the main settings that you can change. See comments in the cell for an explanation of each. Modify them as you see fit and then run all of the cells below to forecast.

*You can press Ctrl+F10 to run all of the cells after the selected one.*

In [24]:
PROJECT_ID = 3349  # 3129 is ID of AI Becnhmarking Pilot project. We kindly ask you not to forecast on any public tournaments or public questions in general
MAX_QUESTIONS_TO_FORECAST = 1_000  # You can set it to some small number for testing or to 1_000_000 to forecast on all available questions
REPREDICT = False # if this is false it won't predict on questions it has previously already predicted on. Set it to true to repredict on all open questions, even if it has made previous predictions.
SUBMIT_FORECASTS = True # If set to False - forecast, but don't submit results to Metaculus platform. If set to True - forecast, and submit results to Metaculus platform
USE_PERPLEXITY_RECENT = True # If set to true the perplexity search used is one that looks for the most recent news on the subject using a GPT prompt completion informed by the forecasting question.
QUESTION_IDS_TO_FORECAST = None # Set to None to disable custom filtering by ID. Set to a list of IDs to only forecast on selected questions, i.e. [24191, 24190, 24189]

# The forecaster weights are used to produce a weighted average of the forecasts. You can adjust these weights to favor a certain forecaster/prompt more heavily.
# Weights should sum to 1.
forecaster1_weight = 0.2
forecaster2_weight = 0.2
forecaster3_weight = 0.2
forecaster4_weight = 0.4

# Prompts
# The prompts used are below, these can be edited to hone your LLM forecasts.
# Here is a glossary of the variables that can be inserted and used in the prompts.
# [[title]]: This is the question text, what shows up at the top of the Metaculus question page
# [[resolution_criteria]]: This is the resolution criteria section of the question, excluding fine print
# [[fine_print]]: This is the fine print section of the question.
# [[background]]: This is the background section of the resolution criteria.
# [[today]]: The current date
# [[forecaster1]] through {forecaster3}: These are the LLM outputs of forecasters 1 through 3, currently being used to feed into the input of forecaster 4 for it to assess in its own forecast.
# [[summary_report]]: This is research from Perplexity. If USE_PERPLEXITY_RECENT is True, this will be the Perplexity info returned when the output of LLM_question_completion is passed to Perplexity.
                  # If USE_PERPLEXITY_RECENT is False and ENABLE_PERPLEXITY_RESEARCH is True, it will return pre-computed Perplexity research on the question stored by Metaculus.
                  # If both are false it will not return anything.

# The LLM_question_completion prompt is used to ask the LLM what question it should ask Perplexity, if using USE_PERPLEXITY_RECENT
LLM_question_completion = """
You're being asked the following forecasting question:

The question is:
[[title]]

And it has these specific resolution details:
[[resolution_criteria]]

Fine print:
[[fine_print]]

To get the latest news that will help you forecast on the question, you need to ask your web search tool one question that would be most valuable to help you forecast
on this question. The question should be posed so that the web search tool will provide you with the most recent information, including the latest information on the progress toward
the criteria in the forecasting question being met. Please complete the sentence below with the most valuable question to ask:

"What is the most recent news available and prior occurrences related to. . . "
"""

# Prompt 1 is used by forecaster 1, with PROMPT_TEMPLATE appended to the end
prompt1 = """
You are a professional forecaster interviewing for a job. The interviewer is also a professional forecaster, with a strong track record of accurate forecasts of the future.
They will ask you a question, and your task is to provide the most accurate forecast you can. To do this, you evaluate past data and trends carefully, make use of comparison classes
of similar events, take into account base rates about how past events unfolded, and outline the best reasons for and against any particular outcome.
You know that great forecasters don't just forecast according to the "vibe" of the question and the considerations.
Instead, they think about the question in a structured way, recording their reasoning as they go,
and they always consider multiple perspectives that usually give different conclusions, which they reason about together.
You can't know the future, and the interviewer knows that, so you do not need to hedge your uncertainty, you are simply trying to give
the most accurate numbers that will be evaluated when the events later unfold.

"""

# Prompt 2 is used by forecaster 2, with PROMPT_TEMPLATE appended to the end
prompt2 = """
You are a professional forecaster being tested on your skill. You are being asked a question for which you must provide the most accurate forecast you can.
Reason step by step about the best forecast, carefully considering the information you are presented with and the potential that some of the
background information and information provided by your research assistant may be out of date or conflicting.
You also know that a key forecasting practice is to consider the outside view, thinking carefully about reference classes and base rates and how often similar events
have happened in the past.

"""

# Prompt 3 is used by forecaster 3, with PROMPT_TEMPLATE appended to the end
prompt3 = """
You are a professional forecaster trying your best to produce an accurate forecast for the question below. You have a keen eye for detail, and know that
the resolution criteria and fine print of a question often contain important edge cases that should be considered. You also know that rookie forecasters
tend to be underconfident, and so your forecasts should be confident and extremize some of the way toward 1% or 99%. You're also aware that forecasting questions tend
to be biased, so when forecasting questions asks about things that would be a departure from the norm or from business as usual, those things tend not to happen.
You will use that knowledge to make the best forecast you can, and describe your reasoning step by step.

"""

# Forecaster 4 uses prompt 4 parts 1 and 2, with PROMPT_TEMPLATE inserted between part 1 and part 2
prompt4part1 = """
You are a professional forecaster trying your best to produce an accurate forecast for the question below.

"""

prompt4part2 = """
Now that you know what the question asks and some relevant background and research, your job is to make the best forecast you can. You know that examining the reasoning of other
forecasters is an excellent way to improve your own forecast. Below I have provided the reasoning from three other forecasters who predicted on the same question.
Examine their reasoning and use it to inform your own, using your expertise as a forecaster to assess which reasoning seems strongest and which seems flawed,
as well as which reasoning seems to incorporate the most accurate information about base rates and historic reference classes. Construct your own reasoning and forecast,
describing your reasoning step by step and incorporating the strongest arguments from the other forecasters in a way that improves your own reasoning. First produce a
one sentence summary of the reasoning of each forecaster (repeating the final probability each predicted), then describe your forecast.

Forecaster A:
[[forecaster1]]

Forecaster B:
[[forecaster2]]

Forecaster C:
[[forecaster3]]
"""

# PROMPT_TEMPLATE is used with the above prompts to share the details about the question with the LLM.
PROMPT_TEMPLATE = """

The question is:
[[title]]

Here are details about how the outcome of the question will be determined, make sure your forecast is consistent with these:
[[resolution_criteria]]

Here is the question's fine print that you need to be consistent with in your forecast:
[[fine_print]]

Here is some background of the question, though note that some of the details may be out of date:
[[background]]

Your research assistant provides the following information that is likely more up to date:
[[summary_report]]

Today is [[today]].

Describe your reasoning step by step and give your final answer as: "Probability: ZZ%", 0-100
"""

## Code

It is recommended that you do not edit the below unless you have coding experience.

Note that currently the LLM is set to gpt-4o, and this is specified in the code below. This can be changed by advanced users, though changing between OpenAI models is much simpler than changing to a model from a different AI organizations.

---



Getting questions (only binaries)

Getting them 10 at a time, you can change offset to "scroll" through them

In [25]:
url = "https://www.metaculus.com/api2/questions/"

params = {
    "has_group": "false",
    "order_by": "-activity",
    "forecast_type": "binary",
    "project": PROJECT_ID,
    "status": "open", # can change this to 'closed' for testing where you're not submitting a forecast, otherwise leave as open
    "type": "forecast",
    "title-and-description-only": "true",
}

In [26]:
def yield_all_questions():
  limit = 10 # This is a page limit, not question limit
  n = 0
  new_questions_found = False

  while True:
    offset = n * limit
    response = requests.get(
        url,
        params={**params, "limit": limit, "offset": offset},
        headers={"Authorization": f"Token {token}"}
    )
    response.raise_for_status()
    questions = response.json().get("results")

    # if repredict is true it will skip to the else and predict on all the questions
    # if repredict is false it will see if "my_predictions" is empty or not for each question, and only predict on questions without a prediction
    if not REPREDICT:
        for question in questions:
            question_id = question['id']

            guess_response = requests.get(
                f"{url}{question_id}/",
                headers={"Authorization": f"Token {token}"}
            )
            guess_response.raise_for_status()

            if not guess_response.json().get("my_predictions"):
                new_questions_found = True
                yield question
    else:
        new_questions_found = True
        yield from questions

    if not response.json().get("next"):
      break
    n += 1

  if not new_questions_found:
        print("No new questions to predict on.")

In [27]:
def find_number_before_percent(s):
    # Use a regular expression to find all numbers followed by a '%'
    matches = re.findall(r'(\d+)%', s)
    if matches:
        # Return the last number found before a '%'
        return int(matches[-1])
    else:
        # Return None if no number found
        return None

In [28]:
# this is used to replace the {} keys with [[]], since sometimes the LLM output uses {} when formatting code.

def replace_keys(text, key_dict, delimiter='[[', end_delimiter=']]'):
    pattern = re.compile(re.escape(delimiter) + '(.*?)' + re.escape(end_delimiter))
    def replace(match):
        key = match.group(1)
        return key_dict.get(key, match.group(0))  # Return the original if key not found
    return pattern.sub(replace, text)

In [29]:
def predict(question_id, prediction_percentage):
  url = f"https://www.metaculus.com/api2/questions/{question_id}/predict/"
  response = requests.post(
      url,
      json={
        "prediction": float(prediction_percentage) / 100
      },
      headers={"Authorization": f"Token {token}"},
  )
  response.raise_for_status()
  print(f"Successfully predicted {prediction_percentage} on question {question_id}")

In [30]:
def formulate_comment(prediction_json):
  comment_blocks = []
  if "reasoning_base_rate" in prediction_json:
    comment_blocks.append("## Base rate estimation")
    comment_blocks.append(prediction_json["reasoning_base_rate"])
  if "reasoning_reference_classes" in prediction_json:
    comment_blocks.append("## Reference classes")
    comment_blocks.append(prediction_json["reasoning_reference_classes"])
  if "reasoning_other" in prediction_json:
    comment_blocks.append("## Additional")
    comment_blocks.append(prediction_json["reasoning_other"])
  return "\n".join(comment_blocks) if comment_blocks else "No reasoning provided"

In [31]:
def comment(question_id, comment_text):

  # for submit_type choose "S" to post regular comment and "N" for private. Tournament submissions should be private comments.
  url = f"https://www.metaculus.com/api2/comments/"
  response = requests.post(
    url,
    json={
      "comment_text":comment_text,"submit_type":"N","include_latest_prediction":True,"question":question_to_forecast['id']
    },
    headers={"Authorization": f"Token {token}"},
  )
  response.raise_for_status()
  print("Comment Success!")

In [32]:
def get_perplexity_research(question_id):
  url = "https://ml.metaculus.com/questions-api/perplexity-research-results/"
  headers = {
    "accept": "application/json",
    "X-API-Key": questions_api_key,
    "content-type": "application/json"
  }
  params = {
    "question_id": question_id
  }
  response = requests.get(url=url, params=params, headers=headers)
  if response.status_code == 404:
    print("No Perplexity research found")
    return "No results found, please use your own knowledge and judgement to forecast"
  content = response.text

  print("Generated research from perplexity:")
  print(content)
  return content

In [33]:
def estimate_pricing(input, output, model):
  encoding = tiktoken.encoding_for_model(model)
  input_len = len(encoding.encode(input))
  output_len = len(encoding.encode(output))


  # hard coding for now, maybe make it smarter later
  # units of $ per token
  gpt4o_input_pricing = 5 / 1_000_000
  gpt4o_output_pricing = 15 / 1_000_000

  input_cost = input_len * gpt4o_input_pricing
  output_cost = output_len * gpt4o_output_pricing
  total_cost = input_cost + output_cost

  return input_cost, output_cost, total_cost

In [34]:
def get_forecast(today, client, question_to_forecast, prompt, summary_report, model):

  title = question_to_forecast["title"]
  resolution_criteria = question_to_forecast["resolution_criteria"]
  background = question_to_forecast["description"]
  if question_to_forecast["fine_print"]:
    fine_print = question_to_forecast["fine_print"]
  else:
    fine_print = "none"

  print("")

  prompt_dict = {
      "title": title,
      "summary_report": summary_report,
      "today": today,
      "background": background,
      "fine_print": fine_print,
      "resolution_criteria": resolution_criteria,
  }

  prompt_content = replace_keys(prompt, prompt_dict)

  print("Here is the prompt used:")
  print(prompt_content)
  print("")

  chat_completion = client.chat.completions.create(
    model=model,
    messages=[
      {
        "role": "user",
        "content": prompt_content
      }
    ]
  )

  gpt_text = chat_completion.choices[0].message.content

  #estimate cost
  input_cost, output_cost, total_cost = estimate_pricing(prompt_content, gpt_text, model)

  # Regular expression to find the number following 'Probability: '
  probability_match = find_number_before_percent(gpt_text)

  # Extract the number if a match is found
  if probability_match:
      probability = int(probability_match) # int(match.group(1))
      print(f"The extracted probability is: {probability}%")
      #probability = min(max(probability, 3), 97) # To prevent extreme forecasts
  else:
      probability = None
      print("No probability found in the text! Skipping!")
  return probability, gpt_text, input_cost, output_cost, total_cost

In [35]:
def call_perplexity(perplexity_prompt):
  model = "llama-3-sonar-large-32k-online"
  perplexity_token_pricing = 1/1_000_000
  perplexity_cost_fixed = 5/1_000

  url = "https://api.perplexity.ai/chat/completions"
  headers = {
    "accept": "application/json",
    "authorization": f"Bearer {perplexity_api_key}",
    "content-type": "application/json"
  }
  payload = {
    "model": model,
    "messages": [
      {
        "role": "system",
        "content": "Be precise and provide all the important details."
      },
      {
        "role": "user",
        "content": perplexity_prompt
      }
    ]
  }
  response = requests.post(url=url, json=payload, headers=headers)
  response.raise_for_status()
  content = response.json()["choices"][0]["message"]["content"]
  print("Generated research from perplexity:")
  print(content)

  # get token and cost estimate

  # currently using the GPT tokenizer with a 1.3 multiplier. Hacky and wrong, but rough estimate.
  # See here for 1.3 factor estimate source: https://github.com/continuedev/continue/issues/878

  multiplier = 1.3
  encoding = tiktoken.encoding_for_model('gpt-4o')
  input_text = perplexity_prompt
  output_text = content
  input_len = len(encoding.encode(input_text)) * multiplier
  output_len = len(encoding.encode(output_text)) * multiplier

  input_cost = input_len * perplexity_token_pricing
  output_cost = output_len * perplexity_token_pricing
  fixed_cost = perplexity_cost_fixed
  total_cost = input_cost + output_cost + perplexity_cost_fixed

  print(f"Total perplexity call cost: ${total_cost}")

  return content, total_cost

In [36]:
def clean_gpt_turbo_markdown(text: str) -> str:
  match = re.search(r"```[\w]+\s+(.*?)\s+```", text, re.DOTALL)
  if match:
    cleaned_text = match.group(1).strip()
  else:
    cleaned_text = text
  return cleaned_text

Forecast on all questions

In [37]:
today = datetime.datetime.now().strftime("%Y-%m-%d")
client = OpenAI()
model = "gpt-4o"
ENABLE_PERPLEXITY_RESEARCH = False  # Previously this could be set to True to get and use pre-computed Perplexity research results for the question, set to False otherwise.
# However, Metaculus no longer supports pre-computed Perplexity research, so it has been permanently set to False here to skip over that setp.

promptset = [prompt1 + PROMPT_TEMPLATE, prompt2 + PROMPT_TEMPLATE, prompt3 + PROMPT_TEMPLATE]

forecasted_count = 0

for question_to_forecast in yield_all_questions():
  if forecasted_count >= MAX_QUESTIONS_TO_FORECAST:
    break
  if QUESTION_IDS_TO_FORECAST is not None and question_to_forecast["id"] not in QUESTION_IDS_TO_FORECAST:
    continue

  print(question_to_forecast["id"], question_to_forecast["title"])

  #define perplexity research to use
  perplexity_total_cost = "N/A"

  if ENABLE_PERPLEXITY_RESEARCH:
    summary_report = get_perplexity_research(question_to_forecast["id"])
  else:
    summary_report = "No results found, please use your own knowledge and judgement to forecast"

  # set summary_report to the perplexity recent search if enabled
  if USE_PERPLEXITY_RECENT:
    #get prompt completion for use with perplexity
    probability, perplexity_recent_prompt, completion_input_cost, completion_output_cost, completion_total_cost = get_forecast(today, client, question_to_forecast, LLM_question_completion, summary_report, model)

    print(f"The completed question posed to perplexity reads: {perplexity_recent_prompt}")
    #get recent news from perplexity
    perplexity_content, perplexity_cost = call_perplexity(perplexity_recent_prompt)

    summary_report = perplexity_content
    perplexity_total_cost = completion_total_cost + perplexity_cost

  # need to iterate through prompts here
  all_forecasts = []
  overall_cost = 0

  for prompt in promptset:
    probability, gpt_text, input_cost, output_cost, total_cost = get_forecast(today, client, question_to_forecast, prompt, summary_report, model)
    all_forecasts.append((probability, gpt_text, input_cost, output_cost, total_cost))
    overall_cost += total_cost
    print(f"Output Reasoning:")
    print(gpt_text)
    print(f"Input cost: ${input_cost}")
    print(f"Output cost: ${output_cost}")
    print(f"Total cost: ${total_cost}")
    print(f"Perplexity search costs (including LLM prompt completion): ${completion_total_cost + perplexity_cost}")
    print("")
    print("~~~~ NEXT PROMPT ~~~~")
    print("")

  prompt4part2_dict = {
      "forecaster1": all_forecasts[0][1],
      "forecaster2": all_forecasts[1][1],
      "forecaster3": all_forecasts[2][1],
  }

  formatted_prompt4part2 = replace_keys(prompt4part2, prompt4part2_dict)

  print("+++++++++++FINAL PROMPT + ++++++++++++++++")

  prompt4 = prompt4part1 + PROMPT_TEMPLATE + formatted_prompt4part2

  probability, gpt_text, input_cost, output_cost, total_cost = get_forecast(today, client, question_to_forecast, prompt4, summary_report, model)

  forecaster1_weight = 0.2
  forecaster2_weight = 0.2
  forecaster3_weight = 0.2
  forecaster4_weight = 0.4

  weighted_forecast = forecaster1_weight*float(all_forecasts[0][0]) + forecaster2_weight*float(all_forecasts[1][0]) + forecaster3_weight*float(all_forecasts[2][0]) + forecaster4_weight*float(probability)
  weighted_forecast = int(weighted_forecast)
  overall_cost = overall_cost + total_cost

  #create summary strings for comments:
  header_string = f"""
  *This forecast is produced from four separate prompts. The first three produce independent forecasts and reasoning using different prompts, and the fourth reads the reasoning and forecasts fo the first three and produces its own forecast. Then a weighted forecast is produced from the four forecasts as described below. The reasoning shown is that of the fourth forecaster or "Summary Forecaster". See [the code](https://colab.research.google.com/drive/1_P7_QNJiJyWBY2qCVu2-_8gVPD1X7mX3?usp=sharing) for more info on the prompts used.*

  * *Model used: {model}*
  * *Weighted formula: ({forecaster1_weight})(Forecaster A) + ({forecaster2_weight})(Forecaster B) + ({forecaster3_weight})(Forecaster C) + ({forecaster4_weight})(Summary Forecaster)*
  * *Estimated cost of model calls based on tokens (excluding Perplexity info used): ${round(overall_cost,3)}*
  * *Estimated (rough) cost of Perplexity call (if using recent Perplexity search, and including LLM prompt completion): ${round(perplexity_total_cost,3)}*
  * ***Final weighted forecast: {weighted_forecast}%***

  ---

  """
  print(header_string)

  forecasted_count += 1
  if SUBMIT_FORECASTS and weighted_forecast is not None:
    predict(question_to_forecast["id"], float(weighted_forecast))
    comment(question_to_forecast["id"], header_string + "PERPLEXITY\n\n" + summary_report + "\n\n---\n\n" + "GPT\n\n" + gpt_text)

  print(f"Output Reasoning:")
  print(gpt_text)
  print("")
  print("FINAL WEIGHTED FORECAST:")
  print(f"{weighted_forecast}%")
  print("")
  print(f"Overall cost was: ${overall_cost}")
  print("")
  print("################ NEXT QUESTION #################")
  print("")

25873 Will the Real-time Sahm Rule Recession Indicator increase Year-over-Year for Aug 2024?

Here is the prompt used:

You're being asked the following forecasting question:

The question is:
Will the Real-time Sahm Rule Recession Indicator increase Year-over-Year for Aug 2024?

And it has these specific resolution details:
This question will resolve based on reporting by the St. Louis Fed here:  https://fred.stlouisfed.org/series/SAHMREALTIME. The reported number for Aug 2024 must be greater than the reported for Aug 2023.

Fine print:
The question will resolve according to the first value published for the period in question, later updates or revisions will be immaterial.

To get the latest news that will help you forecast on the question, you need to ask your web search tool one question that would be most valuable to help you forecast
on this question. The question should be posed so that the web search tool will provide you with the most recent information, including the latest i